In [1]:
import numpy as np
import pandas as pd
import sqlite3

## Import data from tables

In [8]:
mdat = pd.read_csv('genotyping_data/metadata.csv', index_col="dna_chip_id")
mdat = mdat.drop(columns=['Unnamed: 0'])

,breed,sex
dna_chip_id,,
202290551164R09C01,Д,Хр
202341831114R02C01,Д,Хр
202341831114R03C01,Д,Хр
202341831114R04C01,Д,Хр
202290551140R01C01,Д,Хр
...,...,...
203071331169R10C02,Д,Хр
203071331169R12C02,Д,Хр
202341831114R01C01,Д,Хр


In [11]:
mdat.groupby(['breed', 'sex']).count()

,,Unnamed: 0
breed,sex,
Д,Хр,841


Кажется, наша таблица состоит из одних и тех же значений. Но пофиг, возможно, они были бы разными.

In [9]:
genes = pd.read_csv('genotyping_data/genstudio.csv', index_col="Sample ID")
genes = genes.drop(columns=['Unnamed: 0'])
genes

/home/alexey/.conda/envs/py_work/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SNP Name,SNP Index,SNP Aux,SNP,Allele1 - Top,Allele2 - Top,Allele1 - Forward,Allele2 - Forward,Allele1 - AB,Allele2 - AB,Chr,Position,GC Score,GT Score,Theta,R,B Allele Freq,Log R Ratio
Sample ID,,,,,,,,,,,,,,,,,,
202341831114R01C01,1_10573221,1,0,[T/C],-,-,-,-,-,-,1,10573221,0.0000,0.0000,0.942,0.413,1.0000,0.4040
202341831114R01C01,1_10673082,2,0,[T/C],A,A,T,T,A,A,1,10673082,0.8272,0.8076,0.039,0.968,0.0000,0.3017
202341831114R01C01,1_10723065,3,0,[A/G],A,A,T,T,A,A,1,10723065,0.8316,0.8107,0.011,1.577,0.0000,0.0388
202341831114R01C01,1_11337555,4,0,[A/G],A,A,T,T,A,A,1,11337555,0.3781,0.7925,0.045,1.104,0.0000,0.2761
202341831114R01C01,1_11407894,5,0,[A/G],G,G,G,G,B,B,1,11407894,0.9038,0.8670,0.983,1.122,0.9994,0.0022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202341831127R03C02,ALGA0111890,13032,0,[A/G],A,G,T,C,A,B,6,90920481,0.8159,0.8002,0.581,1.572,0.5540,-0.0907
202341831127R03C02,ALGA0111894,13033,0,[A/G],A,G,T,C,A,B,13,73955342,0.9596,0.9298,0.468,1.415,0.4746,0.0558
202341831127R03C02,ALGA0111899,13034,0,[A/G],A,G,A,G,A,B,UWGS,118787518,0.8900,0.8548,0.502,1.282,0.5313,-0.0773


## Create DB, push data to DB

In [12]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection


connect = create_connection(r'snp_sqlite.db')
cursor = connect.cursor()
connect.close()

Connection to SQLite DB successful


In [13]:
! ls

DB.ipynb  genotyping_data  genotyping_data.zip	snp_sqlite.db


In [14]:
connect = create_connection(r'snp_sqlite.db')
mdat.to_sql(name='metadata', con=connect)
genes.to_sql(name='genes', con=connect)
connect.close()

Connection to SQLite DB successful


/home/alexey/.conda/envs/py_work/lib/python3.7/site-packages/pandas/core/generic.py:2786: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


## Commands

### UPDATE

In [53]:
def db_interaction(db, statement):
    con = sqlite3.connect(db)
    cur = con.cursor()
    try:
        cur.execute(statement)
        con.commit()
    except sqlite3.Error as e:
        print("An error occurred:", e.args[0])
    finally:
        con.close()
        
def get_table_from_db(db, table):
    con = sqlite3.connect(db)
    df = pd.read_sql_query(f"SELECT * from {table}", con)
    con.close()
    return df

In [54]:
db_interaction('snp_sqlite.db', 'UPDATE metadata SET sex = "NO" WHERE breed = "Д"')

In [57]:
get_table_from_db('snp_sqlite.db', 'metadata').head()

,dna_chip_id,Unnamed: 0,breed,sex
0,202290551164R09C01,0,Д,NO
1,202341831114R02C01,1,Д,NO
2,202341831114R03C01,2,Д,NO
3,202341831114R04C01,3,Д,NO
4,202290551140R01C01,4,Д,NO


### DELETE

In [58]:
db_interaction('snp_sqlite.db', 'DELETE FROM genes WHERE "Sample ID" = "202341831114R01C01"')

In [59]:
get_table_from_db('snp_sqlite.db', 'genes').head()

,Sample ID,SNP Name,SNP Index,SNP Aux,SNP,Allele1 - Top,Allele2 - Top,Allele1 - Forward,Allele2 - Forward,Allele1 - AB,Allele2 - AB,Chr,Position,GC Score,GT Score,Theta,R,B Allele Freq,Log R Ratio
0,202341831114R01C02,1_10573221,1,0,[T/C],-,-,-,-,-,-,1,10573221,0.0000,0.0000,0.976,0.629,1.0000,1.2718
1,202341831114R01C02,1_10673082,2,0,[T/C],A,A,T,T,A,A,1,10673082,0.8272,0.8076,0.038,0.994,0.0000,0.3418
2,202341831114R01C02,1_10723065,3,0,[A/G],A,A,T,T,A,A,1,10723065,0.8316,0.8107,0.015,1.578,0.0000,0.0379
3,202341831114R01C02,1_11337555,4,0,[A/G],A,A,T,T,A,A,1,11337555,0.3781,0.7925,0.036,1.072,0.0000,0.2582
4,202341831114R01C02,1_11407894,5,0,[A/G],G,G,G,G,B,B,1,11407894,0.9038,0.8670,0.978,1.049,0.9945,-0.0983


### CASCADE

Сначала создадим таблицу с foreign keys

In [135]:
query1 = "PRAGMA foreign_keys=off"
query2 = "BEGIN TRANSACTION"
query3 = 'CREATE TABLE connected_mdat \
(dna_chip_id TEXT REFERENCES genes("Sample ID") ON DELETE CASCADE, \
breed VARCHAR, \
sex VARCHAR)'
query4 = 'INSERT INTO connected_mdat SELECT dna_chip_id, breed, sex FROM metadata'
query5 = "PRAGMA foreign_keys=on"



In [136]:
db_interaction('snp_sqlite.db', query1)
db_interaction('snp_sqlite.db', query2)
db_interaction('snp_sqlite.db', query3)
db_interaction('snp_sqlite.db', query4)
db_interaction('snp_sqlite.db', query5)

Оке, что получилось?

In [137]:
get_table_from_db('snp_sqlite.db', 'connected_mdat').head()

,dna_chip_id,breed,sex
0,202290551164R09C01,Д,Хр
1,202341831114R02C01,Д,Хр
2,202341831114R03C01,Д,Хр
3,202341831114R04C01,Д,Хр
4,202290551140R01C01,Д,Хр


In [139]:
to_remove = '202341831114R02C01'

mdat = get_table_from_db('snp_sqlite.db', 'connected_mdat')
mdat[mdat["dna_chip_id"] == to_remove]

# genes = get_table_from_db('snp_sqlite.db', 'genes')
# genes[genes["Sam"] == to_remove]

,dna_chip_id,breed,sex
1,202341831114R02C01,Д,Хр


In [140]:
db_interaction('snp_sqlite.db', f'DELETE FROM genes WHERE "Sample ID" = "{to_remove}"')

In [141]:
mdat = get_table_from_db('snp_sqlite.db', 'connected_mdat')
mdat[mdat["dna_chip_id"] == to_remove]

,dna_chip_id,breed,sex
1,202341831114R02C01,Д,Хр


А должно было получиться. Печаль((